#### Import the required libraries

In [ ]:
import psycopg2 as psy
import pandas as pd
import json
import os
import time

#### Establish connection with the PostgreSQL database

In [ ]:
database_host = "localhost"
database_name = "dbl_data_challenge"
database_user = "admin"
database_pass = "vZtbqKNXGz27cQCH"

In [ ]:
con = psy.connect(
    host = database_host,
    database = database_name,
    user = database_user,
    password = database_pass
)
cur = con.cursor()

#### Creates the table in which the aba groups will be stored

In [ ]:
cur.execute("""
    CREATE TABLE IF NOT EXISTS aba_groups (id_str VARCHAR(50), aba_id VARCHAR(50), level INT, conversation_id VARCHAR(50), airline_id VARCHAR(50))
""")

#### This gets all the reply pairs from the replies table

In [ ]:
cur.execute("""
    SELECT * FROM replies
""")

data = cur.fetchall()

reply_pairs = {}

for row in data: 
    if row[0] in reply_pairs: 
        reply_pairs[row[0]].append(row[1])
    else: 
        reply_pairs[row[0]] = [row[1]]

#### Airlines that are commented out down below will be skipped, this makes it possible to load airlines in batches. Note that not commenting out an airline will make it so that all its entries are removed from the table, so it will start loading all over again for that airline

In [ ]:
airlines = {
    "KLM": "56377143",
    "AirFrance": "106062176",
    "British_Airways": "18332190",
    "AmericanAir": "22536055",
    "Lufthansa": "124476322",
    "AirBerlin": "26223583",
    "AirBerlin assist": "2182373406",
    "easyJet": "38676903",
    "RyanAir": "1542862735",
    "SingaporeAir": "253340062",
    "Qantas": "218730857",
    "EtihadAirways": "45621423",
    "VirginAtlantic": "20626359"
}

#### The following cells iterate over the airlines put in above: it removes all the aba-groups related to that airline and starts all over inserting values again for that airline, now you can skip airlines that you have already loaded in

In [ ]:
for airline in airlines: 
    start = time.time()
    print("Getting aba groups by "+airline)
    airline_id = airlines[airline]
    cur.execute("""
        DELETE FROM aba_groups WHERE airline_id = %(airline_id)s
    """, {"airline_id": airline_id})
    cur.execute("""
        SELECT id_str, in_reply_to_status_id_str FROM tweets WHERE user_id_str = %(airline_id)s
    """, {"airline_id": airline_id})
    data = cur.fetchall()
    print(str(len(data))+" possible aba groups found")
    for idx, row in enumerate(data): 
        a1 = row[1]
        b = row[0]
        try: 
            a2 = reply_pairs[row[0]]
            cur.execute("""
                SELECT conversation_id FROM conversation_ids WHERE tweet_id = %(a1)s
            """, {"a1": a1})
            conversation_id = cur.fetchall()
            if len(conversation_id) > 0: 
                cur.execute("""
                    INSERT INTO aba_groups (id_str, aba_id, level, conversation_id, airline_id) VALUES (%(id_str)s, %(aba_id)s, %(level)s, %(conversation_id)s, %(airline_id)s)
                """, {"id_str": a1, "aba_id": b, "level": 1, "conversation_id": conversation_id[0], "airline_id": airline_id})
                cur.execute("""
                    INSERT INTO aba_groups (id_str, aba_id, level, conversation_id, airline_id) VALUES (%(id_str)s, %(aba_id)s, %(level)s, %(conversation_id)s, %(airline_id)s)
                """, {"id_str": b, "aba_id": b, "level": 2, "conversation_id": conversation_id[0], "airline_id": airline_id})
                for value in a2: 
                    cur.execute("""
                        INSERT INTO aba_groups (id_str, aba_id, level, conversation_id, airline_id) VALUES (%(id_str)s, %(aba_id)s, %(level)s, %(conversation_id)s, %(airline_id)s)
                    """, {"id_str": value, "aba_id": b, "level": 3, "conversation_id": conversation_id[0], "airline_id": airline_id})
        except KeyError: 
            pass
        if (idx+1) % 500 == 0: 
            print("Currently at "+str(idx+1)+"/"+str(len(data))+" in "+str(time.time()-start)+"s")
            start = time.time()
    con.commit()
    print("Finished loading aba-groups for "+airline)
con.commit()

In [ ]:
cur.close()
con.close()